# <font color ="gold"> Filter & Clean

In [1]:
import matsim
import os
import contextily as ctx
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from my_matsim_utils import *
%matplotlib inline

### <font color ="gold"> Choose Population % to be read

In [2]:
#CHECK THIS FOR THE PATH
pct = 100
place = "Frauenfeld"
# place = "Weinfelden"
# place = "Thurgau"
scenario_name = "BaselineScenario"
simulation_name = "Frauenfeld_Baseline_100pct_11"

In [3]:
import os
import platform

def get_data_folder_path():
    # Get the current operating system
    os_type = platform.system()
    user_name = os.getlogin()

    # Define data folder paths for different systems
    if os_type == 'Windows' and user_name == 'muaa':
        data_folder_path = f"C://Users//{user_name}//Documents//3_MIEI//2023_ABMT_Data//{place}//"    
    elif os_type == 'Linux' and user_name == 'comura':
        data_folder_path = '/cluster/home/comura/.../'
    elif os_type == 'Linux' and user_name == 'cmuratori':
        data_folder_path = '/cluster/home/cmuratori/.../'
    elif os_type == 'Linux' and user_name == 'muaa':
        data_folder_path = '/cluster/home/muaa/.../'
    else:
        raise Exception("Unsupported system configuration")

    return data_folder_path

In [4]:
data_folder_path = get_data_folder_path()
scenario_path = f"{data_folder_path}//{scenario_name}//{pct}pct//"
output_folder_path = f"{data_folder_path}//{simulation_name}//"

### <font color ="gold"> READ ALL THE "0_Analysis" CSVs
Takes around 40 seconds

In [5]:
# Load dataframes that are in common between SynPop and Simulation Outputs
df_households_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_households_synt.csv')
network_geo = gpd.read_file(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/network_geo.geojson')

# Load _synth dataframes
df_activity_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_activity_synt.csv')
# df_plans_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_plans_synt.csv')
df_legs_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_legs_synt.csv')
df_persons_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_persons_synt.csv')
df_routes_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_routes_synt.csv')

# Load _sim dataframes
df_activity_sim = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_activity_sim.csv')
df_legs_sim = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_legs_sim.csv')
df_persons_sim = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_persons_sim.csv')
df_routes_sim = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_routes_sim.csv')

#Load Microcensus dataframes
df_population_mic = pd.read_csv(os.path.join(data_folder_path,'0_Microcensus//population.csv'))
df_trips_mic = pd.read_csv(os.path.join(data_folder_path,'0_Microcensus//trips.csv'))
df_personen_geschlecht = pd.read_csv(os.path.join(data_folder_path,'0_Microcensus//Personen_ZH_Sex.csv'))

C:\Users\muaa\AppData\Local\Temp\ipykernel_20220\3201299563.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_activity_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_activity_synt.csv')
C:\Users\muaa\AppData\Local\Temp\ipykernel_20220\3201299563.py:9: DtypeWarning: Columns (0,4,8,13,14,15,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_persons_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_persons_synt.csv')
C:\Users\muaa\AppData\Local\Temp\ipykernel_20220\3201299563.py:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_routes_synt = pd.read_csv(f'{data_folder_path}/0_PreProcess_CSVs_SyntheticAndOutputs/{pct}pct/df_routes_synt.csv')
C:\Users\muaa\AppData\Local\Temp\ipykernel_20220\3201299563.py:15: DtypeWarning: Columns (0,4,9,14,15,16,17,22) ha

In [6]:
# The network contains information about the coordinate system it is expressed in, which is useful eg. to add map backgrounds
network_geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Comparison Microzensus - Population

### Filtering Data

Filtering Real Population ZH with unknown sex

In [7]:
# Filter out rows where 'SexLang' is 'unbekannt'
df_personen_geschlecht = df_personen_geschlecht[df_personen_geschlecht['SexLang'] != 'unbekannt']

Filtering Cleaning of Synthetic Time Data

In [8]:
def process_time_data(df):
    """
    Function to convert 'dep_time' and 'trav_time' from string to timedelta,
    then to seconds, and calculate 'arrival_time'.
    """
    # Convert 'dep_time' from string to timedelta
    df['dep_time'] = pd.to_timedelta(df['dep_time'])

    # Convert 'dep_time' from timedelta to seconds
    df['departure_time'] = df['dep_time'].dt.total_seconds().astype(int)

    # Convert 'trav_time' from string to timedelta
    df['trav_time'] = pd.to_timedelta(df['trav_time'])

    # Convert 'trav_time' from timedelta to seconds
    df['trav_time_seconds'] = df['trav_time'].dt.total_seconds().astype(int)

    # Calculate 'arrival_time_seconds' by adding 'trav_time_seconds' to 'dep_time_seconds'
    df['arrival_time'] = df['departure_time'] + df['trav_time_seconds']

    return df

# Run the function for both df_legs_synt and df_legs_sim
df_legs_synt = process_time_data(df_legs_synt)
df_legs_sim = process_time_data(df_legs_sim)

In [9]:
# Rename the column 'id' to 'person_id'
df_persons_synt.rename(columns={'id': 'hh_id'}, inplace=True)
df_persons_sim.rename(columns={'id': 'hh_id'}, inplace=True)

In [10]:
# AROUND 21 minutes

def map_person_id_to_activities(df_activities, df_persons, activity_type='Home'):
    """
    Map household IDs from df_persons to df_activities based on home coordinates and 
    propagate the ID to other activities in the same plan.

    :param df_activities: DataFrame containing activities with coordinates (x, y) and plan_id.
    :param df_persons: DataFrame containing person data with home coordinates (home_x, home_y) and hh_id.
    :param activity_type: The type of activity used to map person IDs (default 'Home').
    :return: DataFrame with person_id mapped and propagated.
    """
    # Ensure coordinates are of type float64
    df_activities['x'] = pd.to_numeric(df_activities['x'], errors='coerce')
    df_activities['y'] = pd.to_numeric(df_activities['y'], errors='coerce')
    df_persons['home_x'] = pd.to_numeric(df_persons['home_x'], errors='coerce')
    df_persons['home_y'] = pd.to_numeric(df_persons['home_y'], errors='coerce')

    # Filter df_activities for rows where type is the specified activity type
    home_activities = df_activities[df_activities['type'] == activity_type]

    # Merge the household IDs from df_persons to home_activities based on coordinate match
    merged_home_activities = pd.merge(
        home_activities,
        df_persons[['hh_id', 'home_x', 'home_y']],
        left_on=['x', 'y'],
        right_on=['home_x', 'home_y'],
        how='left'
    )

    # Create a mapping of plan_id to hh_id
    if merged_home_activities['plan_id'].is_unique:
        plan_id_to_hh_id = merged_home_activities.set_index('plan_id')['hh_id']
    else:
        plan_id_to_hh_id = merged_home_activities.groupby('plan_id')['hh_id'].first()

    # Map the hh_id as person_id to all activities in df_activities
    df_activities['person_id'] = df_activities['plan_id'].map(plan_id_to_hh_id)

    # Propagate the person_id to other activities in the same plan
    df_activities['person_id'] = df_activities.groupby('plan_id')['person_id'].transform(lambda x: x.ffill().bfill())

    return df_activities

# Example usage:
df_activity_synt_filtered = map_person_id_to_activities(df_activity_synt, df_persons_synt)
df_activity_sim_filtered = map_person_id_to_activities(df_activity_sim, df_persons_sim)

In [11]:
def process_activity_and_legs_data(df_activity, df_legs, values_to_remove, modes_to_remove):
    # Identify persons with only one 'Home' activity initially
    initial_single_home = df_activity.groupby('person_id').filter(lambda x: len(x) == 1 and x['type'].eq('Home').all())

    # Filter the activity DataFrame
    df_activity_filtered = df_activity[~df_activity['type'].isin(values_to_remove)]

    # Find all 'plan_id' values where 'type' is 'outside'
    plan_ids_to_remove = df_activity_filtered[df_activity_filtered['type'] == 'outside']['plan_id'].unique()

    # Filter out all rows with these 'plan_id' values
    df_activity_filtered = df_activity_filtered[~df_activity_filtered['plan_id'].isin(plan_ids_to_remove)]

    # Additional filter to remove 'outside'
    df_activity_filtered = df_activity_filtered[~df_activity_filtered['type'].isin(['outside'])]

    # Combine 'Access Walk' and 'Egress Walk' into 'Walk' in legs DataFrame
    df_legs['mode'] = df_legs['mode'].replace({'access_walk': 'walk', 'egress_walk': 'walk'})

    # Remove specified modes from the legs DataFrame
    df_legs_filtered = df_legs[~df_legs['mode'].isin(modes_to_remove)]

    # Identify persons who now only have one 'Home' activity
    final_single_home = df_activity_filtered.groupby('person_id').filter(lambda x: len(x) == 1 and x['type'].eq('Home').all())

    # Exclude persons who initially had only one 'Home' activity
    final_single_home = final_single_home[~final_single_home['person_id'].isin(initial_single_home['person_id'])]

    # Remove these persons from the filtered data
    df_activity_filtered = df_activity_filtered[~df_activity_filtered['person_id'].isin(final_single_home['person_id'])]

    return df_activity_filtered, df_legs_filtered

# Define the values to be removed for synthetic and simulated data
values_to_remove = ['freight_unloading', 'freight_loading', 'pt interaction']
modes_to_remove = ['truck', 'outside']

# Process synthetic data
df_activity_synt_filtered, df_legs_synt_filtered = process_activity_and_legs_data(df_activity_synt, df_legs_synt, values_to_remove, modes_to_remove)

# Process simulated data
df_activity_sim_filtered, df_legs_sim_filtered = process_activity_and_legs_data(df_activity_sim, df_legs_sim, values_to_remove, modes_to_remove)


In [12]:
df_activity_synt = df_activity_synt_filtered
df_activity_sim = df_activity_sim_filtered

df_legs_synt = df_legs_synt_filtered
df_legs_sim = df_legs_sim_filtered

Makes New Trips Based on Activity End And Start Points

In [13]:
def create_trips_dataframe(df_activity):
    # List to hold new trip entries
    new_trips = []

    # Iterate over the activity DataFrame
    for i in range(len(df_activity) - 1):
        # Get current and next row
        current_row = df_activity.iloc[i]
        next_row = df_activity.iloc[i + 1]

        # Check if the IDs are consecutive
        if current_row['id'] + 1 == next_row['id']:
            # Create a new trip entry
            new_trips.append({
                'trip_id': current_row['id'],
                'departure_time': current_row['end_time'],
                'arrival_time': next_row['start_time'],
                'start_coor_x': current_row['x'],
                'start_coor_y': current_row['y'],
                'ziel_coor_x': next_row['x'],
                'ziel_coor_y': next_row['y'],
                
            })

    # Create a DataFrame from the list of new trips
    df_trips = pd.DataFrame(new_trips)

    return df_trips


df_trips_synt = create_trips_dataframe(df_activity_synt)
df_trips_sim = create_trips_dataframe(df_activity_sim)

In [14]:
df_trips_synt = df_trips_synt.dropna()
df_trips_sim = df_trips_sim.dropna()

In [15]:
def safe_convert_time(time_str):
    try:
        # Convert to datetime, then to time, and floor to 30-minute bins
        return pd.to_datetime(time_str, format='%H:%M:%S', errors='raise').floor('30T').time()
    except ValueError:
        # Handle invalid time data (e.g., return None or a default time)
        return None

# Apply the conversion function to the DataFrame
df_trips_synt['departure_time'] = df_trips_synt['departure_time'].apply(safe_convert_time)
df_trips_synt['arrival_time'] = df_trips_synt['arrival_time'].apply(safe_convert_time)

df_trips_sim['departure_time'] = df_trips_sim['departure_time'].apply(safe_convert_time)
df_trips_sim['arrival_time'] = df_trips_sim['arrival_time'].apply(safe_convert_time)

In [16]:
# Convert seconds to datetime and resample times to 15-minute bins
df_trips_mic['departure_time'] = pd.to_datetime(df_trips_mic['departure_time'], unit='s').dt.floor('30T').dt.time
df_trips_mic['arrival_time'] = pd.to_datetime(df_trips_mic['arrival_time'], unit='s').dt.floor('30T').dt.time

# # Convert seconds to datetime and resample times to 30-minute bins for df_legs_synt
# df_legs_synt['departure_time'] = pd.to_datetime(df_legs_synt['departure_time'], unit='s').dt.floor('30T').dt.time
# df_legs_synt['arrival_time'] = pd.to_datetime(df_legs_synt['arrival_time'], unit='s').dt.floor('30T').dt.time

In [17]:
# Capitalize and remove underscores from mode names for both dataframes
df_trips_mic['mode'] = df_trips_mic['mode'].str.replace('_', ' ').str.title()
df_legs_synt['mode'] = df_legs_synt['mode'].str.replace('_', ' ').str.title()
df_legs_sim['mode'] = df_legs_sim['mode'].str.replace('_', ' ').str.title()

# Capitalize and remove underscores from type names
df_trips_mic['purpose'] = df_trips_mic['purpose'].str.replace('_', ' ').str.title()
df_activity_synt['type'] = df_activity_synt['type'].str.replace('_', ' ').str.title()
df_activity_sim['type'] = df_activity_sim['type'].str.replace('_', ' ').str.title()

In [18]:
# Remove rows where 'age' is NaN
df_persons_synt = df_persons_synt.dropna(subset=['age'])
df_persons_synt['age'] = df_persons_synt['age'].astype(int)

df_persons_sim = df_persons_sim.dropna(subset=['age'])
df_persons_sim['age'] = df_persons_sim['age'].astype(int)


In [19]:
# Filter out persons younger than 6
df_persons_synt = df_persons_synt[df_persons_synt['age'] >= 6]
df_persons_sim = df_persons_sim[df_persons_sim['age'] >= 6]

In [20]:
# Group all numbers count car larger equal 3 to 3+
def group_cars(value):
    # Convert to integer if the value is a string
    try:
        value_int = int(value)
    except ValueError:
        # Return the value as is if it's not a number
        return value

    # Group all values less than or equal to 3 into '3+'
    if value_int >= 3:
        return '3+'
    else:
        return str(value_int)

# Apply the grouping function to the 'number_of_cars' column
df_population_mic['number_of_cars'] = df_population_mic['number_of_cars'].apply(group_cars)

In [21]:
# Mapping '0' to 'male' and '1' to 'female'
df_population_mic['sex'] = df_population_mic['sex'].replace({0: 'male', 1: 'female'})
df_persons_synt['sex'] = df_persons_synt['sex'].replace({'m': 'male', 'f': 'female'})
df_persons_sim['sex'] = df_persons_sim['sex'].replace({'m': 'male', 'f': 'female'})

Creating Activity Chains

In [22]:
# Function to create activity chains
def create_activity_chain_mic(group):
    chain = '-'.join(['H'] + [purpose[0] for purpose in group['purpose'].tolist()])  # Add 'H' at the start of each chain
    return pd.Series({'activity_chain': chain})

# Function to create uppercase activity chains
def create_activity_chain_syn(group):
    chain = '-'.join([purpose[0].upper() for purpose in group['type'].tolist()])
    return pd.Series({'activity_chain': chain})

# Create activity chains


df_activity_chains_mic =  df_trips_mic.groupby(['person_id']).apply(create_activity_chain_mic).reset_index()
df_activity_chains_syn =  df_activity_synt_filtered.groupby(['plan_id']).apply(create_activity_chain_syn).reset_index()
df_activity_chains_sim =  df_activity_sim_filtered.groupby(['plan_id']).apply(create_activity_chain_syn).reset_index()

In [23]:
# Replace all 'H' with 'H-H' in the 'activity_chain' column
df_activity_chains_syn['activity_chain'] = df_activity_chains_syn['activity_chain'].replace({'H': 'H-H'})
df_activity_chains_sim['activity_chain'] = df_activity_chains_sim['activity_chain'].replace({'H': 'H-H'})
df_activity_chains_mic['activity_chain'] = df_activity_chains_mic['activity_chain'].replace({'H': 'H-H'})

In [24]:
# Merge the 'household_weight' column from df_population_mic to df_trips_mic based on 'person_id'
df_trips_mic = pd.merge(df_trips_mic, df_population_mic[['person_id', 'household_weight']], on='person_id', how='left')

In [25]:
df_trips_mic.columns

Index(['Unnamed: 0', 'HHNR', 'WEGNR', 'f51100', 'f51400', 'wzweck1', 'wzweck2',
       'wmittel', 'S_X_CH1903', 'S_Y_CH1903', 'Z_X_CH1903', 'Z_Y_CH1903',
       'W_X_CH1903', 'W_Y_CH1903', 'w_rdist', 'dauer2', 'mode',
       'mode_detailed', 'purpose', 'departure_time', 'arrival_time',
       'person_id', 'trip_id', 'destination_x', 'destination_y', 'origin_x',
       'origin_y', 'home_x', 'home_y', 'crowfly_distance', 'previous_trip_id',
       'activity_duration', 'parking_cost', 'network_distance', 'origin_point',
       'destination_point', 'household_weight'],
      dtype='object')

### Saving Clean Data

In [26]:
df_trips_mic.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/trips_mic.csv', index=False)
df_trips_synt.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/trips_synt.csv', index=False)
df_trips_sim.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/trips_sim.csv', index=False)

df_activity_chains_syn.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/activity_chains_syn.csv', index=False)
df_activity_chains_sim.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/activity_chains_sim.csv', index=False)
df_activity_chains_mic.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/activity_chains_mic.csv', index=False)

df_population_mic.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/population_clean_mic.csv', index=False)
df_persons_synt.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/population_clean_synth.csv', index=False)
df_persons_sim.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/population_clean_sim.csv', index=False)

df_legs_synt.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/legs_clean_synt.csv', index=False)
df_legs_sim.to_csv(f'{data_folder_path}/1_Cleaned_CSVs/{pct}pct/legs_clean_sim.csv', index=False)